## Loading required libraries 
- Required Libraries:
    - tensorflow > 2
    - numpy
    - pandas
    - sklearn
- Run all cells ( cell -> Run All )

In [1]:
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
import os
import pandas as pd
from tensorflow.keras.layers import Bidirectional
from sklearn.metrics import recall_score, precision_score, f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Load the data in required format
- Specify the base directories of train and test data
- The function **load_dataset** takes the parameter **dataset_type** to decide which dataset to load (train or test)
- The function checks the directory for records, assigns the label to the record as the parent directory (1: wheat, 0: rapeseed) 
- Convert the categorical features in the input data into one hot encodings
- Append each sample to collection of records 
- Return the collection

In [2]:
# load the dataset 

train_dir = 'TimeSen2Crop-SmallSet/train'
test_dir = 'TimeSen2Crop-SmallSet/test'

def load_dataset(dataset_type = None):
    train_x, train_y, test_x = list(), list(), list()
    if(dataset_type in ('train', 'test')):
        load_dir = None
        if(dataset_type == 'train'):
            load_dir = train_dir
            for f in os.listdir(load_dir):
                if(f == 'wheat'):
                    x_dir = os.path.join(load_dir, f)
                    for x_sample in os.listdir(x_dir):
                        X = read_csv(os.path.join(x_dir, x_sample))
                        flag_dummies = pd.get_dummies(X.iloc[:, -1])
                        #print(flag_dummies.shape)
                        #X.iloc[:,-1] = flag_dummies
                        train_x.append(X)
                        train_y.append(1)
                else:
                    x_dir = os.path.join(load_dir, f)
                    for x_sample in os.listdir(x_dir):
                        X = read_csv(os.path.join(x_dir, x_sample))
                        #flag_dummies = pd.get_dummies(X.iloc[:,-1])
                        #X.iloc[:,-1] = flag_dummies
                        train_x.append(X)
                        train_y.append(0)
            return np.array(train_x), np.array(train_y)
        else:
            load_dir = test_dir
            for f in os.listdir(load_dir):
                X = read_csv(os.path.join(load_dir, f))
                test_x.append(X)
            return np.array(test_x)
        
        

## Perform preprocessing , Train and Validation Split
- Cateogical variable, **flag** among various features of input records contains different inputs across samples.
- Collect all the unique values accross inputs, store them into a list and perform one-hot encoding on the list.
- Store the one hot encodings in a dictionary with corresponding flag key. (lookup dictionary)
- Training data is split into training set and validation set.
- The function **preprocess_data** performs following actions:
    - For Each sample in the data provided,
        - the sample is divided into variables containing numerical and categorical features.
        - The numerical part is standardized, The categorical part is converted to one-hot representation by refering to the lookup dictionary.
        - The processed parts are concatenated and the original sample is updated with the processed sample.
- preprocess the training data, validation data

In [3]:
trainX, trainY = load_dataset(dataset_type='train')
test_x = load_dataset(dataset_type='test')

all_flags = set([j for i in trainX for j in i[:, -1]])
all_flags = np.array(list(all_flags))

all_flags_oh = np.array(pd.get_dummies(all_flags))
all_flags_dict = {key: value for (key, value) in zip(all_flags, all_flags_oh)}

trainX = list(trainX)
test_x = list(test_x)
trainY = pd.get_dummies(trainY)

train_x, validation_x, train_y, validation_y = train_test_split(trainX, trainY, test_size=0.1, random_state=42)


def preprocess_data(data):
    for i in range(len(data)):
        sample = data[i]
        sample_n = sample[:, :-1] #all the variables with continous values
        
        stdizer = StandardScaler()
        stdizer = stdizer.fit(sample_n)
        sample_n = stdizer.transform(sample_n)
        
        sample_c = sample[:, -1] # categorical values
        

        sample_c_oh = np.array([all_flags_dict[j] for j in sample_c])
        data[i] = np.concatenate([sample_n, sample_c_oh], axis = 1)
        
        
    data = np.array(data)
    return data
    
train_x_pp = preprocess_data(train_x)
validation_x_pp = preprocess_data(validation_x)


## Predictive Model
- As the data contains **time series sequence**, the problem can be identified as **time series classification**.
- Since **RNNs**, work well for time series applications, the model was built using **LSTMs**.
- The model is essentially a **Bidirectional Stacked LSTM Neural Network**
- Network architecture is as follows:
    - input - (time steps, features)
    - Bidirectional LSTM layer with 70 LSTM units.
    - Dropout layer with p = 0.5.
    - Batch Normalization Layer
    - Bidirectional LSTM layer with 70 LSTM units.
    - Dropout layer with p = 0.5.
    - Batch Normalization Layer
    - Fully connected layer with 100 neurons
    - Relu activation layer.
    - output - Fully connected layer with 2 neurons for classification with Softmax activation function.
- Loss function: Categorical Cross Entropy Loss
- Optimizer : Adam
- Although it is a binary classification problem, 2 units in output layer were used with softmax instead of one unit simply because of the simplicity. 

In [4]:
def lstm_model(lstm_units, train_x, train_y):
    n_timesteps, n_features = train_x.shape[1], train_x.shape[2]
    
    model = Sequential()
    
    model.add(Bidirectional(LSTM(lstm_units, return_sequences=True), input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(tf.keras.layers.BatchNormalization())
    
    model.add(Bidirectional(LSTM(lstm_units)))
    model.add(Dropout(0.5))
    model.add(tf.keras.layers.BatchNormalization())
    
    model.add(Dense(100, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(Dense(2, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model
    



## Training and Evaluation

- The model is trained with following hyper parameters:
    - epochs : 20, batch size: 32
    
- We perform training until we get the desired accuracy and we set a loop threshold as well, to prevent infinite loop.

- The **predict_crop_type** function is used to output the crop type given the dataset and trained model as parameters.

- The model is evaulated based on **accuracy**, **precision**, **recall** and **f1-score** metrics on the validation dataset.



In [5]:
lstm_model = lstm_model(70, train_x_pp, train_y)
accuracy = 0
loop_count = 0
loop_threshold = 10
accuracy_threshold = 0.95
while(accuracy < accuracy_threshold and loop_count < loop_threshold):
    lstm_model.fit(train_x_pp, train_y, epochs=20, batch_size=32, verbose=0)
    _, temp_acc = lstm_model.evaluate(validation_x_pp, validation_y, verbose=1)
    accuracy = temp_acc
    loop_count += 1
    

print(f'Final validation accuracy {accuracy}')

1/1 [==============================] - 1s 832ms/step - loss: 0.2216 - accuracy: 0.9565
Final validation accuracy 0.95652174949646


In [6]:
def predict_crop_type(x, model):
    predictions = list()
    predictions.append(model.predict(x))
    return predictions

crop_type = {0:'rapeseed', 1:'wheat'}

In [7]:
predictions = predict_crop_type(validation_x_pp, lstm_model)
predictions = np.argmax(predictions, axis = 2)[0]
predictions_labels = np.array([str(crop_type[i]) for i in predictions])

validation_y_np = validation_y.to_numpy()
ground_truth = np.argmax(validation_y_np, axis= 1)
ground_truth_labels = np.array([str(crop_type[i]) for i in ground_truth])

precision = precision_score(ground_truth, predictions)
recall = recall_score(predictions, ground_truth)
f1_score = f1_score(predictions, ground_truth)

print(f'precision: {np.round(precision, decimals=2)}, recall: {np.round(recall, decimals=2)} f1 score: {np.round(f1_score, decimals=2)}')

precision: 0.92, recall: 0.92 f1 score: 0.96


## Comparison of predicted labels and ground truth labels
- The prediction is performed on validation data
- The table below shows the predictions compared to the ground truth.

In [8]:
comparison_matrix = np.stack([predictions_labels, ground_truth_labels], axis = 1)
comparison_matrix = pd.DataFrame(comparison_matrix, columns=['predcitions', 'ground truth'])
comparison_matrix



,predcitions,ground truth
0,wheat,wheat
1,wheat,wheat
2,rapeseed,rapeseed
3,wheat,wheat
4,wheat,wheat
5,wheat,wheat
6,rapeseed,rapeseed
7,rapeseed,rapeseed
8,rapeseed,rapeseed
9,rapeseed,rapeseed


## Prediction on the test data set
- preprocessing is performed on the test data
- Model predicts the outcomes
- The table shows the predictions for the test data

In [9]:
test_x_pp = preprocess_data(test_x)

In [10]:
predictions_test = predict_crop_type(test_x_pp, lstm_model)
predictions_test = np.argmax(predictions_test, axis = 2)[0]
predictions_test_labels = np.array([str(crop_type[i]) for i in predictions_test])

file_names_test = np.array(os.listdir(test_dir))
prediction_table = np.stack([file_names_test, predictions_test_labels], axis = 1)
prediction_table_df = pd.DataFrame(prediction_table, columns=['Sample', 'Predicted Label'])
prediction_table_df

,Sample,Predicted Label
0,7.csv,wheat
1,10.csv,wheat
2,4.csv,wheat
3,2.csv,wheat
4,9.csv,wheat
5,8.csv,wheat
6,1.csv,wheat
7,5.csv,wheat
8,6.csv,wheat
9,3.csv,wheat


## Since almost all of the samples are predicted as ***wheat***, we can assume that the test data belongs to ***wheat*** class.